In [2]:
%%writefile app6.py
import streamlit as st
import numpy as np
import pandas as pd
import joblib
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

# -------------------------------------------
# Load Trained Model and Scaler
# -------------------------------------------
model = load_model("lstm_model.keras")
scaler = joblib.load("scaler.pkl")

# -------------------------------------------
# Streamlit Page Setup
# -------------------------------------------
st.set_page_config(page_title="📈 LSTM Stock Forecast Dashboard", layout="wide")
st.title("📊 LSTM Stock Price Prediction & 30-Day Forecast")
st.markdown("Upload your stock CSV to visualize **actual vs predicted** prices and forecast the next 30 days 📅")

# -------------------------------------------
# File Upload
# -------------------------------------------
uploaded_file = st.file_uploader("📂 Upload CSV (must contain 'Close' column)", type=["csv"])

if uploaded_file:
    df = pd.read_csv(uploaded_file)

    st.subheader("📄 Dataset Preview")
    st.dataframe(df.head())

    # Validate column
    if "Close" not in df.columns:
        st.error("❌ CSV must include a 'Close' column (closing stock price).")
    else:
        # -------------------------------------------
        # Prepare Data
        # -------------------------------------------
        data = df["Close"].values.reshape(-1, 1)
        try:
            scaled_data = scaler.transform(data)
        except Exception as e:
            st.error(f"Scaler error: {e}")

        seq_length = 60
        X_test = [scaled_data[i - seq_length:i, 0] for i in range(seq_length, len(scaled_data))]
        X_test = np.array(X_test).reshape(-1, seq_length, 1)

        if X_test.shape[0] == 0:
            st.error(f"Not enough rows — need at least {seq_length} data points.")
        else:
            # -------------------------------------------
            # Model Predictions
            # -------------------------------------------
            predictions = model.predict(X_test)
            predictions = scaler.inverse_transform(predictions)
            true_values = df["Close"].values[seq_length:]

            # -------------------------------------------
            # Performance Metrics
            # -------------------------------------------
            rmse = sqrt(mean_squared_error(true_values, predictions))
            mae = mean_absolute_error(true_values, predictions)

            st.subheader("📉 Model Performance")
            col1, col2 = st.columns(2)
            col1.metric("RMSE (Root Mean Square Error)", f"{rmse:.2f}")
            col2.metric("MAE (Mean Absolute Error)", f"{mae:.2f}")

            # -------------------------------------------
            # Actual vs Predicted Graph
            # -------------------------------------------
            st.subheader("📊 Actual vs Predicted Stock Prices")
            fig, ax = plt.subplots(figsize=(12, 5))
            ax.plot(true_values, label="Actual", color="deepskyblue", linewidth=2)
            ax.plot(predictions, label="Predicted", color="orange", linestyle="--", linewidth=2)
            ax.set_title("LSTM Model — Actual vs Predicted Close Prices", fontsize=14)
            ax.set_xlabel("Days")
            ax.set_ylabel("Price")
            ax.legend()
            ax.grid(True, linestyle="--", alpha=0.5)
            st.pyplot(fig)

            # -------------------------------------------
            # Future Forecast (Next 1–30 Days)
            # -------------------------------------------
            st.subheader("🔮 30-Day Future Price Forecast")
            days = st.slider("Select number of future days", 1, 30, 30)

            last_60_days = scaled_data[-60:]
            future_predictions = []
            current_input = last_60_days.copy()

            for _ in range(days):
                X_input = current_input.reshape(1, 60, 1)
                next_pred = model.predict(X_input)
                future_predictions.append(next_pred[0, 0])
                current_input = np.append(current_input[1:], next_pred, axis=0)

            # Inverse scale
            future_predictions = scaler.inverse_transform(np.array(future_predictions).reshape(-1, 1))

            # Create Day vs Price table
            future_df = pd.DataFrame({
                "Day": np.arange(1, days + 1),
                "Predicted Close Price": future_predictions.flatten()
            })

            st.dataframe(future_df.style.highlight_max(axis=0, color='lightgreen'))
            st.markdown("📅 **Above table shows predicted stock prices for upcoming days (Day 1 → Day 30).**")

            # -------------------------------------------
            # Forecast Graph (Real Market Style)
            # -------------------------------------------
            fig2, ax2 = plt.subplots(figsize=(12, 5))

            # Plot past 100 days actual data
            ax2.plot(range(len(true_values[-100:])),
                     true_values[-100:],
                     label="Recent Actual",
                     color="deepskyblue",
                     linewidth=2)

            # Plot future forecast continuation
            ax2.plot(range(len(true_values[-100:]), len(true_values[-100:]) + days),
                     future_predictions.flatten(),
                     label="Future Forecast",
                     color="limegreen",
                     linewidth=2.5,
                     linestyle="-")

            # Chart styling
            ax2.set_title(f"Next {days} Days Stock Price Forecast", fontsize=14)
            ax2.set_xlabel("Day", fontsize=12)
            ax2.set_ylabel("Predicted Stock Price", fontsize=12)
            ax2.grid(True, linestyle="--", alpha=0.5)
            ax2.legend()
            st.pyplot(fig2)

            st.markdown(
                "> 🟢 **Interpretation Tip:** The green line shows predicted movement for upcoming days — "
                "this helps visualize short-term market trend forecasts similar to professional stock analysis tools."
            )

else:
    st.info("👆 Please upload a valid stock CSV to begin analysis.")

# -------------------------------------------
# Footer
# -------------------------------------------
st.markdown("---")
st.markdown("<center>🧠 Built with TensorFlow LSTM | Visualized using Streamlit | © 2025</center>", unsafe_allow_html=True)


Overwriting app6.py


In [ ]:
# streamlit run app6.py